In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from model import PIC
import cv2
from utils import read_config
import dataset
import os

# Prepare data

In [2]:
cfg = read_config('configs/base_config.yaml')

In [3]:
if cfg.DATASET == 'ImageNet_noise_reduction':
    train_dataset = dataset.ImageNet_noise_reduction(cfg)
    train_dataloader = DataLoader(dataset=train_dataset, batch_size=cfg.BATCH_SIZE, shuffle=True, num_workers=8)

    val_dataset = dataset.ImageNet_noise_reduction(cfg, 'Val')
    val_dataloader = DataLoader(dataset=val_dataset, batch_size=cfg.BATCH_SIZE, shuffle=True, num_workers=8)
    
print('Train dataset length: ', len(train_dataset))
print('Val dataset length: ', len(val_dataset))

Train dataset length:  77657
Val dataset length:  13000


# Prepare net

In [4]:
from model import Model
net = Model(cfg)
net.to(torch.device(net.cfg.DEVICE))

RuntimeError: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero.

# Train

In [5]:
from train import train

In [6]:
train(net, train_dataloader)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: p-k-babkin (p-k-babkin-rsm). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/petr/.netrc


Training epoch [1/10] ...


  0%|          | 0/7766 [00:00<?, ?it/s]


RuntimeError: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero.

# Look at results

In [7]:
from utils import reconstruct_im
import matplotlib.pyplot as plt

In [8]:
sample = train_dataset[0]
test_inp = {
    'im_orig' : torch.tensor(sample['im_orig']).to(torch.float),
    'im_noisy' : torch.tensor(sample['im_noisy']).to(torch.float),
}

net.eval()
net.cpu()
res = net(test_inp)

In [9]:
reconstructed_im = reconstruct_im(test_inp['im_orig'], cfg)

cv2.imshow('sss', reconstructed_im)

cv2.waitKey(0)

cv2.destroyAllWindows()

In [10]:
reconstruct_im(test_inp['im_orig'], cfg).shape

(200, 200, 3)

In [11]:
from metrics import calc_metrics

calc_metrics(reconstruct_im(test_inp['im_orig'], cfg), reconstruct_im(test_inp['im_noisy'], cfg))

{'MSE': 105.48525833333333,
 'PSNR': 27.898885900666638,
 'SSIM': 0.25272413966557433}